# House Sale Price Analysis

This notebook does x y z


### Imports and Functions

In [15]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

In [34]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# Added after the session - a better metrics function!
def print_metrics(y_tr, tr_pred, y_te, te_pred, log=False):
    '''
    Prints the R2 Score, Mean Absolute Error and Root Mean Squared Error
    Will unlog to get MAE & RMSE in terms of the original target if log=True
    
    Inputs:
        y_tr: array-like or pandas series
            Actual target values for the train set
        tr_pred: array-like or pandas series
            Predicted target values for the train set
        y_te: array-like or pandas series
            Actual target values for the test set
        te_pred: array-like or pandas series
            Predicted target values for the test set
        log: boolean
            Toggles whether the target values have been logged or not
            If True, assumes all other arguments passed into the function have been logged
            
    Outputs:
        None, just prints the metrics
    '''
    # Unlogging all variables if you set log=True
    if log == True:
        # Please note - if you used log to log the variables, change this to exp
        y_tr_unlog = np.expm1(y_tr)
        tr_pred_unlog = np.expm1(tr_pred)
        y_te_unlog = np.expm1(y_te)
        te_pred_unlog = np.expm1(te_pred)
    
    # Printing train scores
    print("Training Scores")
    print("-"*10)
    print(f"R2: {r2_score(y_tr, tr_pred):.4f}") # R2 should not be done on unlogged values
    if log == True:
        print(f"RMSE: {mean_squared_error(y_tr_unlog, tr_pred_unlog, squared=False):.4f}")
        print(f"MAE: {mean_absolute_error(y_tr_unlog, tr_pred_unlog):.4f}")
    else:
        print(f"RMSE: {mean_squared_error(y_tr, tr_pred, squared=False):.4f}")
        print(f"MAE: {mean_absolute_error(y_tr, tr_pred):.4f}")
    
    print("\n"+"*"*10)
    
    # Printing test scores
    print("Testing Scores")
    print("-"*10)
    print(f"R2: {r2_score(y_te, te_pred):.4f}") # R2 should not be done on unlogged values
    if log == True:
        print(f"RMSE: {mean_squared_error(y_te_unlog, te_pred_unlog, squared=False):.4f}")
        print(f"MAE: {mean_absolute_error(y_te_unlog, te_pred_unlog):.4f}")
    else:
        print(f"RMSE: {mean_squared_error(y_te, te_pred, squared=False):.4f}")
        print(f"MAE: {mean_absolute_error(y_te, te_pred):.4f}")   

### Initial Data Understanding

In [2]:
df = pd.read_csv("data/kc_house_data.csv")

In [3]:
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NaN,0.0,...,7,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650
1,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,0.0,0.0,...,7,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639
2,5631500400,2/25/2015,180000.0,2,1.00,770,10000,1.0,0.0,0.0,...,6,770,0.0,1933,NaN,98028,47.7379,-122.233,2720,8062
3,2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,0.0,0.0,...,7,1050,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000
4,1954400510,2/18/2015,510000.0,3,2.00,1680,8080,1.0,0.0,0.0,...,8,1680,0.0,1987,0.0,98074,47.6168,-122.045,1800,7503


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21597 non-null  int64  
 1   date           21597 non-null  object 
 2   price          21597 non-null  float64
 3   bedrooms       21597 non-null  int64  
 4   bathrooms      21597 non-null  float64
 5   sqft_living    21597 non-null  int64  
 6   sqft_lot       21597 non-null  int64  
 7   floors         21597 non-null  float64
 8   waterfront     19221 non-null  float64
 9   view           21534 non-null  float64
 10  condition      21597 non-null  int64  
 11  grade          21597 non-null  int64  
 12  sqft_above     21597 non-null  int64  
 13  sqft_basement  21597 non-null  object 
 14  yr_built       21597 non-null  int64  
 15  yr_renovated   17755 non-null  float64
 16  zipcode        21597 non-null  int64  
 17  lat            21597 non-null  float64
 18  long  

In [5]:
df.describe()

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
count,2.159700e+04,2.159700e+04,21597.000000,21597.000000,21597.000000,2.159700e+04,21597.000000,19221.000000,21534.000000,21597.000000,21597.000000,21597.000000,21597.000000,17755.000000,21597.000000,21597.000000,21597.000000,21597.000000,21597.000000
mean,4.580474e+09,5.402966e+05,3.373200,2.115826,2080.321850,1.509941e+04,1.494096,0.007596,0.233863,3.409825,7.657915,1788.596842,1970.999676,83.636778,98077.951845,47.560093,-122.213982,1986.620318,12758.283512
std,2.876736e+09,3.673681e+05,0.926299,0.768984,918.106125,4.141264e+04,0.539683,0.086825,0.765686,0.650546,1.173200,827.759761,29.375234,399.946414,53.513072,0.138552,0.140724,685.230472,27274.441950
min,1.000102e+06,7.800000e+04,1.000000,0.500000,370.000000,5.200000e+02,1.000000,0.000000,0.000000,1.000000,3.000000,370.000000,1900.000000,0.000000,98001.000000,47.155900,-122.519000,399.000000,651.000000
25%,2.123049e+09,3.220000e+05,3.000000,1.750000,1430.000000,5.040000e+03,1.000000,0.000000,0.000000,3.000000,7.000000,1190.000000,1951.000000,0.000000,98033.000000,47.471100,-122.328000,1490.000000,5100.000000
50%,3.904930e+09,4.500000e+05,3.000000,2.250000,1910.000000,7.618000e+03,1.500000,0.000000,0.000000,3.000000,7.000000,1560.000000,1975.000000,0.000000,98065.000000,47.571800,-122.231000,1840.000000,7620.000000
75%,7.308900e+09,6.450000e+05,4.000000,2.500000,2550.000000,1.068500e+04,2.000000,0.000000,0.000000,4.000000,8.000000,2210.000000,1997.000000,0.000000,98118.000000,47.678000,-122.125000,2360.000000,10083.000000
max,9.900000e+09,7.700000e+06,33.000000,8.000000,13540.000000,1.651359e+06,3.500000,1.000000,4.000000,5.000000,13.000000,9410.000000,2015.000000,2015.000000,98199.000000,47.777600,-121.315000,6210.000000,871200.000000


In [6]:
df.loc[df['bedrooms'] == 33]

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
15856,2402100895,6/25/2014,640000.0,33,1.75,1620,6000,1.0,0.0,0.0,...,7,1040,580.0,1947,0.0,98103,47.6878,-122.331,1330,4700


In [7]:
df = df.loc[df['bedrooms'] < 33]

In [8]:
df.describe()

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
count,2.159600e+04,2.159600e+04,21596.000000,21596.000000,21596.000000,2.159600e+04,21596.000000,19220.000000,21533.000000,21596.000000,21596.000000,21596.000000,21596.000000,17754.000000,21596.000000,21596.000000,21596.000000,21596.000000,21596.000000
mean,4.580575e+09,5.402920e+05,3.371828,2.115843,2080.343165,1.509983e+04,1.494119,0.007596,0.233874,3.409752,7.657946,1788.631506,1971.000787,83.641489,98077.950685,47.560087,-122.213977,1986.650722,12758.656649
std,2.876764e+09,3.673760e+05,0.904114,0.768998,918.122038,4.141355e+04,0.539685,0.086827,0.765702,0.650471,1.173218,827.763251,29.375460,399.957185,53.514040,0.138552,0.140725,685.231768,27275.018316
min,1.000102e+06,7.800000e+04,1.000000,0.500000,370.000000,5.200000e+02,1.000000,0.000000,0.000000,1.000000,3.000000,370.000000,1900.000000,0.000000,98001.000000,47.155900,-122.519000,399.000000,651.000000
25%,2.123049e+09,3.220000e+05,3.000000,1.750000,1430.000000,5.040000e+03,1.000000,0.000000,0.000000,3.000000,7.000000,1190.000000,1951.000000,0.000000,98033.000000,47.471100,-122.328000,1490.000000,5100.000000
50%,3.904930e+09,4.500000e+05,3.000000,2.250000,1910.000000,7.619000e+03,1.500000,0.000000,0.000000,3.000000,7.000000,1560.000000,1975.000000,0.000000,98065.000000,47.571800,-122.231000,1840.000000,7620.000000
75%,7.308950e+09,6.450000e+05,4.000000,2.500000,2550.000000,1.068550e+04,2.000000,0.000000,0.000000,4.000000,8.000000,2210.000000,1997.000000,0.000000,98118.000000,47.678000,-122.125000,2360.000000,10083.000000
max,9.900000e+09,7.700000e+06,11.000000,8.000000,13540.000000,1.651359e+06,3.500000,1.000000,4.000000,5.000000,13.000000,9410.000000,2015.000000,2015.000000,98199.000000,47.777600,-121.315000,6210.000000,871200.000000


### FSM

In [17]:
y = df['price']

used_cols = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'condition', 'grade', 'zipcode']
X = df[used_cols]

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [19]:
scaler = StandardScaler()

scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [20]:
lr = LinearRegression()

lr.fit(X_train_scaled, y_train)

LinearRegression()

In [21]:
train_preds = lr.predict(X_train_scaled)
test_preds = lr.predict(X_test_scaled)

In [54]:
# sketch version of a metrics function coded out during the session

# def metrics(y_train, train_preds, y_test, test_preds):
#     print('Train Scores:')
#     print(f'R2: {r2_score(y_train, train_preds)}')
#     print(f'RMSE: {mean_squared_error(y_train, train_preds, squared=False)}')
#     print('Test Scores:')
#     print(f'R2: {r2_score(y_test, test_preds)}')
#     print(f'RMSE: {mean_squared_error(y_test, test_preds, squared=False)}') 

In [24]:
print_metrics(y_train, train_preds, y_test, test_preds)

Training Scores
----------
R2: 0.5688
RMSE: 240314.8716
MAE: 156284.5511

**********
Testing Scores
----------
R2: 0.5605
RMSE: 246349.4286
MAE: 156143.2444


In [25]:
test_preds.min()

-268547.18845424545

In [26]:
test_preds.max()

2591692.432412645

In [27]:
y_test

3686      132500.0
10247     415000.0
4037      494000.0
3437      355000.0
19291     606000.0
           ...    
8476      343000.0
18276     397000.0
15003     615000.0
13478    1330000.0
18399     784500.0
Name: price, Length: 5399, dtype: float64

In [28]:
test_preds

array([ 142257.84027487,  333516.42000677,  319072.47964895, ...,
        557501.73372277, 1052604.12379231, 1608624.92486253])

In [29]:
results = y_test.reset_index()

In [30]:
results['predicted_price'] = test_preds

In [31]:
results.head()

,index,price,predicted_price
0,3686,132500.0,142257.840275
1,10247,415000.0,333516.420007
2,4037,494000.0,319072.479649
3,3437,355000.0,323176.808432
4,19291,606000.0,365319.501065


In [32]:
results.sort_values(by='predicted_price').drop('price', axis=1)

,index,predicted_price
3640,15698,-2.685472e+05
4718,8588,-1.738350e+05
196,12063,-1.620613e+05
336,7369,-1.565677e+05
1369,8614,-1.547766e+05
...,...,...
3123,1162,2.136859e+06
1944,13398,2.325111e+06
3426,3910,2.465116e+06
4415,14542,2.469748e+06


In [33]:
X_test

,bedrooms,bathrooms,sqft_living,sqft_lot,condition,grade,zipcode
3686,3,0.75,850,8573,3,6,98146
10247,3,1.00,1510,6083,4,6,98115
4037,4,2.25,1790,42000,3,7,98045
3437,2,1.50,1140,2500,3,7,98106
19291,3,1.00,1500,3920,3,7,98107
...,...,...,...,...,...,...,...
8476,3,1.00,1410,18600,5,7,98059
18276,5,1.00,1170,6757,4,6,98125
15003,3,1.75,1670,5100,5,7,98144
13478,4,2.25,3260,4640,5,9,98112


### Log Price

In [35]:
y_train_log = np.log1p(y_train)
y_test_log = np.log1p(y_test)

In [36]:
lr_log = LinearRegression()

lr_log.fit(X_train_scaled, y_train_log)

LinearRegression()

In [37]:
train_preds_log = lr_log.predict(X_train_scaled)
test_preds_log = lr_log.predict(X_test_scaled)

In [38]:
test_preds_log

array([12.46444604, 12.6889219 , 12.75209644, ..., 13.07734308,
       13.78730668, 14.50915745])

In [39]:
print_metrics(y_train_log, train_preds_log, y_test_log, test_preds_log, log=True)

Training Scores
----------
R2: 0.5880
RMSE: 249466.3568
MAE: 145527.2459

**********
Testing Scores
----------
R2: 0.5792
RMSE: 231938.1411
MAE: 145044.8212


In [40]:
np.expm1(y_test_log)

3686      132500.0
10247     415000.0
4037      494000.0
3437      355000.0
19291     606000.0
           ...    
8476      343000.0
18276     397000.0
15003     615000.0
13478    1330000.0
18399     784500.0
Name: price, Length: 5399, dtype: float64

In [41]:
y_test

3686      132500.0
10247     415000.0
4037      494000.0
3437      355000.0
19291     606000.0
           ...    
8476      343000.0
18276     397000.0
15003     615000.0
13478    1330000.0
18399     784500.0
Name: price, Length: 5399, dtype: float64

In [42]:
np.expm1(test_preds_log)

array([ 258963.44192959,  324136.11441022,  345273.98758973, ...,
        477988.03824939,  972189.13750526, 2000998.66283722])